In [ ]:
!pip install tensorboard

In [ ]:
import json
from datetime import datetime
import torch.nn as nn

from args import get_parser
from utils import *
from mtad_gat import MTAD_GAT
from prediction import Predictor
from training import Trainer

In [ ]:
class Args:
    def __init__(self):
        # Data params
        self.dataset = "IVECO"
        self.resample_rate = 1.0
        self.cut = 1.0
        self.scaler = 'minmax'
        self.train_test_split = 0.7
        self.no_anomaly_train = True
        self.group = "1-1"
        self.lookback = 100
        self.normalize = True
        self.spectral_residual = False
        self.spec_res = False

        # Model params
        self.kernel_size = 7
        self.use_gatv2 = True
        self.feat_gat_embed_dim = None
        self.time_gat_embed_dim = None
        self.gru_n_layers = 1
        self.gru_hid_dim = 150
        self.fc_n_layers = 3
        self.fc_hid_dim = 150
        self.recon_n_layers = 1
        self.recon_hid_dim = 150
        self.alpha = 0.2

        # Train params
        self.epochs = 30
        self.val_split = 0.1
        self.bs = 256
        self.init_lr = 1e-3
        self.shuffle_dataset = True
        self.dropout = 0.3
        self.use_cuda = True
        self.print_every = 1
        self.log_tensorboard = True

        # Predictor params
        self.scale_scores = False
        self.use_mov_av = False
        self.gamma = 1.0
        self.level = None
        self.q = None
        self.dynamic_pot = False

        # Other
        self.comment = ""




In [ ]:
id = datetime.now().strftime("%d%m%Y_%H%M%S")

parser = get_parser()
args = Args()

dataset = args.dataset
window_size = args.lookback # default 100
spec_res = args.spec_res 
normalize = args.normalize # default True
n_epochs = args.epochs #default 30
batch_size = args.bs # default 256
init_lr = args.init_lr
val_split = args.val_split
shuffle_dataset = args.shuffle_dataset #default true
use_cuda = args.use_cuda
print_every = args.print_every
log_tensorboard = args.log_tensorboard
group_index = args.group[0]
index = args.group[2:]
args_summary = str(args.__dict__)
print(args_summary)

if dataset == 'SMD':
    output_path = f'output/SMD/{args.group}'
    (x_train, _), (x_test, y_test) = get_data(f"machine-{group_index}-{index}", normalize=normalize)
elif dataset in ['MSL', 'SMAP', 'SWAT', 'SKAB', 'WADI','METRO', 'ACT', 'IVECO']:
    output_path = f'output/{dataset}'
    (x_train, _), (x_test, y_test) = get_data(dataset, normalize=normalize)
else:
    raise Exception(f'Dataset "{dataset}" not available.')

log_dir = f'{output_path}/logs'
if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
save_path = f"{output_path}/{id}"

x_train = torch.from_numpy(x_train).float()
x_test = torch.from_numpy(x_test).float()
n_features = x_train.shape[1]

target_dims = get_target_dims(dataset)
if target_dims is None:
    out_dim = n_features
    print(f"Will forecast and reconstruct all {n_features} input features")
elif type(target_dims) == int:
    print(f"Will forecast and reconstruct input feature: {target_dims}")
    out_dim = 1
else:
    print(f"Will forecast and reconstruct input features: {target_dims}")
    out_dim = len(target_dims)

train_dataset = SlidingWindowDataset(x_train, window_size, target_dims)
test_dataset = SlidingWindowDataset(x_test, window_size, target_dims)

train_loader, val_loader, test_loader = create_data_loaders(
    train_dataset, batch_size, val_split, shuffle_dataset, test_dataset=test_dataset
)

model = MTAD_GAT(
    n_features,
    window_size,
    out_dim,
    kernel_size=args.kernel_size,
    use_gatv2=args.use_gatv2,
    feat_gat_embed_dim=args.feat_gat_embed_dim,
    time_gat_embed_dim=args.time_gat_embed_dim,
    gru_n_layers=args.gru_n_layers,
    gru_hid_dim=args.gru_hid_dim,
    forecast_n_layers=args.fc_n_layers,
    forecast_hid_dim=args.fc_hid_dim,
    recon_n_layers=args.recon_n_layers,
    recon_hid_dim=args.recon_hid_dim,
    dropout=args.dropout,
    alpha=args.alpha
)

optimizer = torch.optim.Adam(model.parameters(), lr=args.init_lr)
forecast_criterion = nn.MSELoss()
recon_criterion = nn.MSELoss()

trainer = Trainer(
    model,
    optimizer,
    window_size,
    n_features,
    target_dims,
    n_epochs,
    batch_size,
    init_lr,
    forecast_criterion,
    recon_criterion,
    use_cuda,
    save_path,
    log_dir,
    print_every,
    log_tensorboard,
    args_summary
)

trainer.fit(train_loader, val_loader)

plot_losses(trainer.losses, save_path=save_path, plot=False)

# Check test loss
test_loss = trainer.evaluate(test_loader)
print(f"Test forecast loss: {test_loss[0]:.5f}")
print(f"Test reconstruction loss: {test_loss[1]:.5f}")
print(f"Test total loss: {test_loss[2]:.5f}")

# Some suggestions for POT args
level_q_dict = {
    "SMAP": (0.90, 0.005),
    "MSL": (0.90, 0.001),
    "SWAT": (0.90, 0.001),
    "WADI": (0.90, 0.001),
    "METRO": (0.90, 0.001),
    "ACT": (0.90, 0.001),
    "SKAB": (0.90, 0.001),
    "SMD-1": (0.9950, 0.001),
    "SMD-2": (0.9925, 0.001),
    "SMD-3": (0.9999, 0.001),
    'IVECO': (0.90, 0.001)
}
key = "SMD-" + args.group[0] if args.dataset == "SMD" else args.dataset
level, q = level_q_dict[key]
if args.level is not None:
    level = args.level
if args.q is not None:
    q = args.q

# Some suggestions for Epsilon args
reg_level_dict = {"SMAP": 0,"SWAT": 0,"WADI": 0, 'IVECO': 0, "METRO": 0,"ACT": 0,"SKAB":0, "MSL": 0, "SMD-1": 1, "SMD-2": 1, "SMD-3": 1}
key = "SMD-" + args.group[0] if dataset == "SMD" else dataset
reg_level = reg_level_dict[key]

trainer.load(f"{save_path}/model.pt")
prediction_args = {
    'dataset': dataset,
    "target_dims": target_dims,
    'scale_scores': args.scale_scores,
    "level": level,
    "q": q,
    'dynamic_pot': args.dynamic_pot,
    "use_mov_av": args.use_mov_av,
    "gamma": args.gamma,
    "reg_level": reg_level,
    "save_path": save_path,
}
best_model = trainer.model
predictor = Predictor(
    best_model,
    window_size,
    n_features,
    prediction_args,
)

label = y_test[window_size:] if y_test is not None else None
predictor.predict_anomalies(x_train, x_test, label)

# Save config
args_path = f"{save_path}/config.txt"
with open(args_path, "w") as f:
    json.dump(args.__dict__, f, indent=2)